In [1]:
import numpy as np
import pandas as pd

# Load data into DataFrame
df = pd.read_csv('/kaggle/input/synthetic-spect-xy/synthetic_spectra_xy.csv')

# Access columns from DataFrame
wavelengths = df['Wavelength']
intensities = df['Intensity']

# 1. Identify Baseline Regions
def identify_baseline_regions(spectrum, min_peak_height=0.01):
    # Implement algorithm to identify baseline regions in spectrum
    # Parameters: 
    # - spectrum: Raman spectrum as a numpy array of intensity values
    # - min_peak_height: Minimum peak height to consider a point as a peak
    # Returns: list of tuples specifying start & end indices of baseline regions
    
    # Find local minima in spectrum
    is_peak = (spectrum < np.roll(spectrum, 1)) & (spectrum < np.roll(spectrum, -1))

    baseline_regions = []
    region_start = 0

    for i in range(1, len(spectrum) - 1):
        if is_peak[i]:
            # If previous region is long enough, consider it a baseline region
            if i - region_start > 2:
                baseline_regions.append((region_start, i))
            region_start = i + 1

    # Check if the last region is a baseline region
    if region_start < len(spectrum) - 1:
        baseline_regions.append((region_start, len(spectrum) - 1))

    # Filter regions based on peak height
    baseline_regions = [region for region in baseline_regions if np.min(spectrum[region[0]:region[1] + 1]) < min_peak_height]

    return baseline_regions

# Apply baseline identification function to intensities array
baseline_regions = identify_baseline_regions(intensities)

# 2. Fit Chebyshev Polynomials to Baseline Regions
for start, end in baseline_regions:
    # Get the wavelengths and intensities in the baseline region
    baseline_wavelengths = wavelengths[start:end]
    baseline_intensities = intensities[start:end]
    
    # Fit Chebyshev polynomial to the baseline region
    poly_order = 3  # Choose an appropriate order for the Chebyshev polynomial
    coeffs = np.polynomial.chebyshev.chebfit(baseline_wavelengths, baseline_intensities, poly_order)

    # 3. Update the Fitted Curve 
    # implement a loop here to update the fitted polynomial to ensure it remains beneath the spectrum.

    # 4. Subtract the Fitted Polynomial
    # Evaluate the fitted polynomial over the entire spectrum range
    fitted_baseline = np.polynomial.chebyshev.chebval(wavelengths, coeffs)

    # Subtract fitted baseline from original spectrum to correct baseline
    corrected_spectrum = intensities - fitted_baseline

/opt/conda/lib/python3.10/site-packages/numpy/polynomial/chebyshev.py:1671: RankWarning: The fit may be poorly conditioned
  return pu._fit(chebvander, x, y, deg, rcond, full, w)
